In [ ]:
# !pip install -U "transformers[torch]" "datasets>=2.16.0" "accelerate>=0.26.0" \
# "evaluate" "pandas>=2.0.0" "matplotlib" "scikit-learn" "umap-learn" "huggingface_hub"

In [ ]:

# hide_output
from datasets import load_dataset

emotions = load_dataset("emotion")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:

emotions

In [ ]:
train_ds = emotions["train"]
train_ds

In [ ]:
train_ds[1]['text']

0: sadness (เศร้า)
1: joy (ดีใจ/มีความสุข)
2: love (รัก)
3: anger (โกรธ)
4: fear (กลัว)
5: surprise (ประหลาดใจ)


In [ ]:

print(train_ds.features)

print(train_ds[:5])

In [ ]:
import pandas as pd

emotions.set_format(type="pandas")
df = emotions["train"][:]
df.head()

In [ ]:
def label_int2str(row):
    return emotions["train"].features["label"].int2str(row)

df["label_name"] = df["label"].apply(label_int2str)
df.head()

In [ ]:

import matplotlib.pyplot as plt

df["label_name"].value_counts(ascending=True).plot.barh()
plt.title("Frequency of Classes")
plt.show()

In [ ]:
emotions.reset_format()

In [ ]:
# hide_output
from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:

text = "Tokenizing text is a core task of NLP."
tokenized_text = list(text)
print(tokenized_text)

In [ ]:
token2idx = {ch: idx for idx, ch in enumerate(sorted(set(tokenized_text)))}
print(token2idx)

In [ ]:
tokenized_text = text.split()
print(tokenized_text)

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [ ]:
print(tokenize(emotions["train"][:2]))

เติม Special Token  เพื่อ

บอกโครงสร้างประโยค, ควบคุมพฤติกรรมการเรียนรู้ของโมเดล, ทำให้โมเดล ไม่เห็นเฉลยทั้งหมด ตอนเทรน โดย [unk] =unknown, [CLS] = token แรกของทุก sequence [SEP] = คั่นประโยค [Mask] = ซ่อยคำเพื่อไม่เห็นเฉลยทั้งหมด

In [ ]:
#hide_input
tokens2ids = list(zip(tokenizer.all_special_tokens, tokenizer.all_special_ids))
data = sorted(tokens2ids, key=lambda x : x[-1])
df = pd.DataFrame(data, columns=["Special Token", "Special Token ID"])
df.T

เข้ารหัส token ทั้งชุดข้อมูล

In [ ]:
# hide_output
emotions_encoded = emotions.map(tokenize, batched=True, batch_size=None)

In [ ]:
print(emotions_encoded["train"].column_names)

attention_mask คือแผนที่บอกโมเดลว่า token ไหน มีตัวตนจริง  1 คือใช้ 0 คือไม่ใช้ กัน padding token

In [ ]:
emotions_encoded["train"][0]

โหลดโมเดล

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoModel

model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

ทดสอบดูขนาดของข้อมูล Tensor (batch,sequence_Length) ที่เป็น 6 เพราะเติม Token พิเศษ [CLS] this is a test [SEP]

In [ ]:

text = "this is a test"
inputs = tokenizer(text, return_tensors="pt")
print(f"Input tensor shape: {inputs['input_ids'].size()}")
print(tokenizer.decode(inputs["input_ids"][0]))

เอา input ผ่าน encoder ของโมเดลเรียบร้อยแล้ว และได้ vector ออกมา

In [ ]:
inputs = {k:v.to(device) for k,v in inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)
print(outputs)

In [ ]:
outputs.last_hidden_state.size()

In [ ]:

outputs.last_hidden_state[:,0].size()

ดึงค่า hidden state ของแต่ละประโยคออกมาแล้วใส่ใน json เดิมใช่ไหม ชื่อตัวแปรhidden state

In [ ]:
def extract_hidden_states(batch):
    # Place model inputs on the GPU
    inputs = {k:v.to(device) for k,v in batch.items()
              if k in tokenizer.model_input_names}
    # Extract last hidden states
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    # Return vector for [CLS] token
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

In [ ]:
emotions_encoded.set_format("torch",
                            columns=["input_ids", "attention_mask", "label"])

เวกเตอร์ + UMAP + linear probe = การวัด representation ก่อน fine-tune เพื่อ วัดศักยภาพ representation แทนการ validate ตรง ๆ

In [ ]:
#hide_output
emotions_hidden = emotions_encoded.map(extract_hidden_states, batched=True)

In [ ]:
emotions_hidden["train"].column_names
print(emotions_hidden["train"][0])

การทำ representation analysis + visualization

In [ ]:

import numpy as np

X_train = np.array(emotions_hidden["train"]["hidden_state"])
X_valid = np.array(emotions_hidden["validation"]["hidden_state"])
y_train = np.array(emotions_hidden["train"]["label"])
y_valid = np.array(emotions_hidden["validation"]["label"])
X_train.shape, X_valid.shape

ใช้ UMAP ในการลดมิติ hidden state แล้วพล็อตดูว่า class แต่ละอัน แยกกลุ่มกันได้ดีแค่ไหน X คือค่าแกนที่ 1 Y คือค่าแกนที่ 2

In [ ]:
from umap import UMAP
from sklearn.preprocessing import MinMaxScaler

# Scale features to [0,1] range
X_scaled = MinMaxScaler().fit_transform(X_train)
# Initialize and fit UMAP
mapper = UMAP(n_components=2, metric="cosine").fit(X_scaled)
# Create a DataFrame of 2D embeddings
df_emb = pd.DataFrame(mapper.embedding_, columns=["X", "Y"])
df_emb["label"] = y_train
df_emb.head()

แผนที่การกระจายตัวของ sentence embeddings แยกตาม class

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(7,5))
axes = axes.flatten()
cmaps = ["Greys", "Blues", "Oranges", "Reds", "Purples", "Greens"]
labels = emotions["train"].features["label"].names

for i, (label, cmap) in enumerate(zip(labels, cmaps)):
    df_emb_sub = df_emb.query(f"label == {i}")
    axes[i].hexbin(df_emb_sub["X"], df_emb_sub["Y"], cmap=cmap,
                   gridsize=20, linewidths=(0,))
    axes[i].set_title(label)
    axes[i].set_xticks([]), axes[i].set_yticks([])

plt.tight_layout()
plt.show()

ทำการเทรนตัวจำแนกประเภทเพื่อเรียนรู้ความสัมพันธ์ระหว่าง sentence embedding ของแต่ละประโยคกับคลาสอารมณ์ที่กำหนด

In [ ]:
#hide_output
# We increase `max_iter` to guarantee convergence
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(max_iter=3000)
lr_clf.fit(X_train, y_train)

In [ ]:

lr_clf.score(X_valid, y_valid)

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
dummy_clf.score(X_valid, y_valid)

จาก confusion matrix พบว่า เมื่อใช้ pretrained embedding ร่วมกับ Logistic Regression ผลลัพธ์ยังมีการสับสนระหว่างคลาสค่อนข้างมาก อย่างไรก็ตาม หลังจากทำการ fine-tuning โมเดล Transformer กับข้อมูลอารมณ์โดยตรง ความสามารถในการจำแนกคลาสดีขึ้นอย่างชัดเจน

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()

y_preds = lr_clf.predict(X_valid)
plot_confusion_matrix(y_preds, y_valid, labels)

ทำการ Fine-Tuned โมเดลด้วยข้อมูลดาต้าเซต

In [ ]:
# hide_output
from transformers import AutoModelForSequenceClassification

num_labels = 6
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=num_labels)
         .to(device))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

กำหนด parameter ที่ใช้ในการเทรน

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(emotions_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"

training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    eval_strategy="epoch",      
    disable_tqdm=False,
    logging_steps=logging_steps,
    push_to_hub=False,          
    log_level="error"
)

ทำการ Fine Tuned

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=emotions_encoded["train"],
                  eval_dataset=emotions_encoded["validation"],
                  tokenizer=tokenizer)
trainer.train();

ทดสอบกับชุดข้อมูล Validation Set

In [ ]:
# hide_output
preds_output = trainer.predict(emotions_encoded["validation"])

In [ ]:
preds_output.metrics

In [ ]:
y_preds = np.argmax(preds_output.predictions, axis=1)

In [ ]:
plot_confusion_matrix(y_preds, y_valid, labels)


ตรวจสอบผลลัพธ์รายประโยค  หลังการ fine-tuning

In [ ]:
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):
    # Place all input tensors on the same device as the model
    inputs = {k:v.to(device) for k,v in batch.items()
              if k in tokenizer.model_input_names}

    with torch.no_grad():
        output = model(**inputs)
        pred_label = torch.argmax(output.logits, axis=-1)
        loss = cross_entropy(output.logits, batch["label"].to(device),
                             reduction="none")

    # Place outputs on CPU for compatibility with other dataset columns
    return {"loss": loss.cpu().numpy(),
            "predicted_label": pred_label.cpu().numpy()}

In [ ]:
# hide_output
# Convert our dataset back to PyTorch tensors
emotions_encoded.set_format("torch",
                            columns=["input_ids", "attention_mask", "label"])
# Compute loss values
emotions_encoded["validation"] = emotions_encoded["validation"].map(
    forward_pass_with_label, batched=True, batch_size=16)

เอาผลการทำนายรายประโยคหลัง fine-tuning มาอยู่ในรูป DataFrame เพื่อทำ error analysis

In [ ]:

emotions_encoded.set_format("pandas")
cols = ["text", "label", "predicted_label", "loss"]
df_test = emotions_encoded["validation"][:][cols]
df_test["label"] = df_test["label"].apply(label_int2str)
df_test["predicted_label"] = (df_test["predicted_label"]
                              .apply(label_int2str))

In [ ]:
#hide_output
df_test.sort_values("loss", ascending=False).head(10)

In [ ]:
#hide_output
df_test.sort_values("loss", ascending=True).head(10)

Push Model ขึ้น Hugging Face

In [ ]:
#hide_output
trainer.push_to_hub(commit_message="Training completed!")

ทำการ Download โมเดลจาก  Hugging Face ที่ Deploy ไป

In [ ]:

from transformers import pipeline


model_id = "Posathip/distilbert-base-uncased-finetuned-emotion"
classifier = pipeline("text-classification", model=model_id)

ทดสอบผลลัพธ์ของประโยค

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


emotions = ["sadness", "joy", "love", "anger", "fear", "surprise"]

custom_tweet = "I very enjoy in this class."
preds = classifier(custom_tweet, return_all_scores=True)

print(f"Tweet: {custom_tweet}")


for i, pred in enumerate(preds[0]):
    label_name = emotions[i]
    print(f"{label_name}: {pred['score']*100:.2f}%")

print("-" * 20)


preds_df = pd.DataFrame(preds[0])
preds_df["label"] = emotions

plt.bar(preds_df["label"], 100 * preds_df["score"], color='C0')
plt.title(f'"{custom_tweet}"')
plt.ylabel("Class probability (%)")
plt.show()